# Get friends (following)

- **Input:** List of screen names
- **Output:** List of friends of those screen names, together with other account information

## Setup, Twitter tokens, inputs

In [ ]:
from configparser import ConfigParser
import json
import pickle
import time

import pandas as pd
import tweepy

from random import randint


# Accounts input
USERS_FILENAME = "data/firstvoters-samples116-info-with-labels.xlsx"
USERS_SHEET = "samples"
SCREEN_NAME_COL = "screen_name"
ID_COL = "id_str"


# Read Twitter API keys and tokens from a config file
config_object = ConfigParser()
config_object.read("config.ini")
twitter_auth = config_object["TWITTER_AUTH"]

consumer_key = twitter_auth["CONSUMER_KEY"]
consumer_secret = twitter_auth["CONSUMER_SECRET"]
access_token = twitter_auth["ACCESS_TOKEN"]
access_token_secret = twitter_auth["ACCESS_TOKEN_SECRET"]


def print_progress(i: int):
    if (i+1) % 10 == 0:
        print("/", end="")
    elif (i+1) % 5 == 0:
        print(",", end="")
    else:
        print(".", end="")


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True, retry_count=1, retry_delay=2)

users_df = pd.read_excel(USERS_FILENAME, sheet_name=USERS_SHEET)

ids = users_df[ID_COL].to_list()
screen_names = users_df[SCREEN_NAME_COL].to_list()

del users_df

print(f"Screen names: {screen_names[:2]} .. {screen_names[-2:]}")
print(f"Total: {len(screen_names):,}")

In [ ]:
# In case we already fetched some of the nodes, don't repeat it

#EXISTING_NODES_FILENAME_PKL = "data/samples-friends-120nodes.pkl"
#existing_nodes = {}
#with open(EXISTING_NODES_FILENAME_PKL, "rb") as file:
#    existing_nodes = pickle.load(file)
#existing_screen_names = set(existing_nodes.keys())
#
#new_screen_names = list(set(screen_names) - existing_screen_names)
#
#print(f"New screen names: {new_screen_names[:2]} .. {new_screen_names[-2:]}")
#print(f"Total: {len(new_screen_names):,}")
#
#screen_names = new_screen_names

## Get friends (following) of each user

In [ ]:
# Wait to get full remaining calls
def to_sleep(reset: int):
    secs_to_sleep = reset - int(time.time())
    if secs_to_sleep > 180:
        secs_to_sleep = 180 + randint(1, 20)
    return secs_to_sleep

job_len = len(screen_names)

rate_limit_status = api.rate_limit_status()["resources"]["friends"]["/friends/list"]
rate_limit = rate_limit_status["limit"]
remaining = rate_limit_status["remaining"]
limit_reset = rate_limit_status["reset"]

while remaining < rate_limit - 1 and limit_reset < 60:
    secs_to_sleep = to_sleep(limit_reset)
    print(f"Calls remaining: {remaining}. Sleeping for: {secs_to_sleep}")
    time.sleep(secs_to_sleep)

    rate_limit_status = api.rate_limit_status()["resources"]["friends"]["/friends/list"]
    remaining = rate_limit_status["remaining"]
    limit_reset = rate_limit_status["reset"]

# Start collecting
friends = {}
for i, name in enumerate(screen_names):
    print(f"{i+1:3}/{job_len} Screen name: {name} |", end="")

    _friends = []
    try:
        cursor = tweepy.Cursor(api.friends, screen_name=name)
        for p, page in enumerate(cursor.pages()):
            _friends.extend(page)
            print_progress(p)
            time.sleep(randint(58, 62))
    except tweepy.TweepError as e:
        print(f"Error: {e.api_code}")
        print(e.reason)
    else:
        print(f"| Done.")
        friends[name] = _friends

    time.sleep(randint(1, 3))

len(friends)

## Save data to file

In [ ]:
FRIENDS_FILENAME_PKL = "data/samples-friends.pkl"

print(f"Save: {len(friends):,} entries")
with open(FRIENDS_FILENAME_PKL, "wb") as file:
    pickle.dump(friends, file)